In [38]:
%run "./5. CNN Definition.ipynb"

X: (800, 360, 25)
Y: (800,)


In [39]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import StratifiedKFold

In [40]:
model_dir = "models"

num_samples = 800
num_features = 360
seq_length = 25
num_classes = 2

num_epochs = 10
num_folds = 5

learning_rate = 0.001
batch_size = 32

In [41]:
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [42]:
k_sampler = StratifiedKFold(n_splits=num_folds, shuffle=True)

# Create a SummaryWriter object
writer = SummaryWriter()

for fold_index, (train_indices, test_indices) in enumerate(k_sampler.split(X, Y)):
    print(f"Fold {fold_index + 1}:")

    np.save(f'./models/model_{fold_index + 1}_train.npy', train_indices)
    np.save(f'./models/model_{fold_index + 1}_test.npy', test_indices)

    train_length = len(train_indices)
    test_length = len(test_indices)
    x_train, y_train = X[train_indices], Y[train_indices]
    x_test, y_test = X[test_indices], Y[test_indices]

    model = MultiFeatureTimeSeries().float()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Model Training:
    for epoch_index in range(num_epochs):
        print(f"\tEpoch {epoch_index}:")
        train_loss = 0.0

        model.train()
        for batch_index in range(0, train_length, batch_size):
            inputs = torch.from_numpy(x_train[batch_index: batch_index + batch_size]).float()
            labels = torch.from_numpy(y_train[batch_index: batch_index + batch_size]).long()

            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / (train_length // batch_size)
        print(f"\t\tTrain Loss: {avg_train_loss:.4f}")

        # Write training loss to TensorBoard
        writer.add_scalar('Loss/Train', avg_train_loss, fold_index * num_epochs + epoch_index)

        val_loss = 0.0
        correct = 0
        total = 0

        # Model Evaluation:
        model.eval()
        with torch.no_grad():
            inputs = torch.from_numpy(x_test).float()
            labels = torch.from_numpy(y_test).long()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Compute accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Print validation loss and accuracy
        print(f"\t\tValidation Loss: {val_loss / test_length:.4f}")
        print(f"\t\tValidation Accuracy: {(100 * correct / total):.2f}%")

        # Write validation loss and accuracy to TensorBoard
        writer.add_scalar('Loss/Validation', val_loss / test_length, fold_index * num_epochs + epoch_index)
        writer.add_scalar('Accuracy/Validation', 100 * correct / total, fold_index * num_epochs + epoch_index)
    
    # Save the model.
    torch.save(model, f'./models/model_{fold_index + 1}.pth')

# Close the SummaryWriter
writer.close()


Fold 1:
	Epoch 0:
		Train Loss: 0.6961
		Validation Loss: 0.0042
		Validation Accuracy: 56.25%
	Epoch 1:
		Train Loss: 0.5973
		Validation Loss: 0.0034
		Validation Accuracy: 88.12%
	Epoch 2:
		Train Loss: 0.4391
		Validation Loss: 0.0026
		Validation Accuracy: 89.38%
	Epoch 3:
		Train Loss: 0.3385
		Validation Loss: 0.0021
		Validation Accuracy: 93.75%
	Epoch 4:
		Train Loss: 0.2799
		Validation Loss: 0.0019
		Validation Accuracy: 95.00%
	Epoch 5:
		Train Loss: 0.2424
		Validation Loss: 0.0017
		Validation Accuracy: 96.25%
	Epoch 6:
		Train Loss: 0.2143
		Validation Loss: 0.0016
		Validation Accuracy: 97.50%
	Epoch 7:
		Train Loss: 0.1907
		Validation Loss: 0.0015
		Validation Accuracy: 97.50%
	Epoch 8:
		Train Loss: 0.1698
		Validation Loss: 0.0014
		Validation Accuracy: 97.50%
	Epoch 9:
		Train Loss: 0.1506
		Validation Loss: 0.0013
		Validation Accuracy: 97.50%
Fold 2:
	Epoch 0:
		Train Loss: 0.6928
		Validation Loss: 0.0043
		Validation Accuracy: 56.25%
	Epoch 1:
		Train Loss: 0.5